In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os 

# Set directory
Path = os.getcwd()
path = Path.replace(os.sep, '/')

# %%


In [25]:
df.index = pd.to_datetime(df.index, format="%a, %d.%m. %H:%M")
type(df.index)
df.index = df.index.map(lambda x:x.replace(year = 2022))

In [26]:
df = pd.read_excel(file, skiprows= 16, index_col='Time')

In [27]:
file = 'npro_kfw55_2.xlsx'

df = pd.read_excel(file, skiprows= 16, index_col='Time', usecols=[1,2,3])
# df.index = pd.date_range(start='01.01.2022 00:00', end='31.12.2022 23:45', freq='h')

df.index = pd.to_datetime(df.index, format="%a, %d.%m. %H:%M")
df.index = df.index.map(lambda x:x.replace(year = 2022))

df_load = df.resample('15min').interpolate(method='linear')
df_load['Heat Demand [kW]'] = df_load['Space heating (kW)'] + df_load['Domestic hot water (kW)']


In [28]:

file_polysun = 'Polysun_raw.csv'

df_polysun = pd.read_csv(file_polysun, index_col='Datum', sep=';', encoding='iso8859_2')
df_polysun.index = pd.date_range(start='01.01.2022 00:00', end='31.12.2022 23:45', freq='15min')

rename_dict = {
    'Wetterdaten: Mittlere Aussentemperatur [°C] (Tamb)': 'T_amb',
    # 'Gebäude Space heating: demand 443 Mhw: Heizwärmebedarf [W] (Qdem)': 'Heat Demand SH [W]',
    # 'Gebäude Space heating: demand 443 Mhw: Soll-Raumtemperatur [°C]': 'T_room_set',
    'Elektrische Verbraucher Elec. Demand: 425 MWh: Elektrischer Verbrauch [W] (Ecs)': 'Electricity Demand [W]',
    'Wetterdaten: Normale Direktstrahlung  [W/m˛] (Bn)': 'DNI',
    'Wetterdaten: Diffusstrahlung, Jahressumme [W/m˛] (Dh)': 'diffuse_radiation',
    'Wetterdaten: Globalstrahlung, Jahressumme [W/m˛] (Gh)': 'global_radiation',
    # 'Warmwasserbedarf 343 MWh: Domestic hot water: Energiebedarf [W] (Qdem)': 'Heat Demand DHW [W]',
    'Wetterdaten: Windgeschwindigkeit [m/s] (Vwnd)': 'wind_speed'
}

# Subset and rename columns
df_polysun.columns = df_polysun.columns.to_series().map(rename_dict)
df_polysun = df_polysun.loc[:, df_polysun.columns.notna()] # delete all columns with nan headers
# df_polysun.DNI = pd.to_numeric(df_polysun.DNI)

df_polysun.head()

,DNI,diffuse_radiation,global_radiation,T_amb,Electricity Demand [W],wind_speed
2022-01-01 00:00:00,0,0.0,0.0,11.5,49358,2.3
2022-01-01 00:15:00,0,0.0,0.0,11.5,51043,2.3
2022-01-01 00:30:00,0,0.0,0.0,11.5,51228,2.3
2022-01-01 00:45:00,0,0.0,0.0,11.5,52631,2.3
2022-01-01 01:00:00,0,0.0,0.0,11.7,45575,1.7


In [29]:
df_merge = df_load.join(df_polysun)

In [30]:
df_merge.to_csv('Input_kfw55_2.csv', header='HEATLOAD', index = True) #Still not able to have a header row, and index title

In [14]:
# #To add an additional timestamp column!!

# df = pd.read_csv('Input_kfw55_2.csv', skiprows=1)
# df['Timestamp'] = df['Time']
# df.index = df['Time']
# df.drop(columns='Time', inplace=True)
# df.to_csv('Input_kfw55_2.csv', header='HEATLOAD', index = True)